In [60]:
from hungarian import Hungarian

In [61]:
matrix = [[-30,-40,-45,-35,-30],
          [-35,-45,-60,-30,-50],
          [-50,-70,-65,-50,-30],
          [-45,-25,-50,-60,-60]]

In [62]:
cost_matrix = [[30,40,45,35,30],
               [35,45,60,30,50],
               [50,70,65,50,30],
               [45,25,50,60,60]]

In [63]:
#takole se pozene najcenejse prirejanje
hungarian = Hungarian(matrix)
hungarian.calculate()

In [64]:
#takole pa najdrazje (najvecji dobicek)
hungarian = Hungarian(cost_matrix, is_profit_matrix=True)
hungarian.calculate()

In [65]:
result = hungarian.get_results()
potential = hungarian.get_total_potential()

In [66]:
result

[(2, 1), (0, 2), (1, 4), (3, 3)]

In [67]:
potential

225

# Megaron

In [68]:
#izberi dan, od kdaj do kdaj, dobi datume,dobi datume

In [69]:
import time
import datetime
import pandas as pd
import numpy as np

In [72]:
def min_max_timestamp(leto):
    min_timestamp = datetime.datetime(leto,1,1).timestamp()
    max_timestamp = datetime.datetime(leto,12,31).timestamp()
    return (min_timestamp,max_timestamp)

def min_max_timestamp_schoolyear(leto_zacetek):
    '''nekako studentsko solsko leto'''
    min_timestamp = (datetime.datetime(leto_zacetek,10,1).timestamp())
    max_timestamp = (datetime.datetime(leto_zacetek+1,6,25).timestamp())
    return (min_timestamp,max_timestamp)
def gen_days(min_timestamp, max_timestamp, iso_weekday):
    '''creates a list of every day between timestamps, that is the weekday (1 monday, 7 sunday)'''
    
    day = 60*60*24
    week = day*7
    
    min_weekday = datetime.datetime.fromtimestamp(min_timestamp).isoweekday()
    diff = iso_weekday-min_weekday
    if (diff < 0):
        diff += 7
    
    timestamp_list = []
    timestamp  = min_timestamp + diff*day
    while(timestamp <= max_timestamp):
        timestamp_list.append(timestamp)
        timestamp += week
    return timestamp_list

In [75]:
a = min_max_timestamp_schoolyear(2017)

In [76]:
datetime.datetime.now()

datetime.datetime(2018, 9, 11, 9, 19, 38, 407160)

In [77]:
my_list = gen_days(a[0],a[1],2)

In [139]:
my_list[-1]
dates = pd.DataFrame(my_list).

In [136]:
#preberemo rojstne dneve
df = pd.read_csv('lanski.csv', names = ['name','date'])

In [137]:
#dan = pd.Timestamp(df['datum'])
leto = datetime.datetime.now().year
df['day'] = df['date'].astype(str).str[:2]
df['month'] = df['date'].astype(str).str[3:5]
df['year'] = np.where((df['month'].astype(int) < 7),leto+1,leto)
df['timestamp'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df[['name', 'timestamp']].copy()

In [138]:
df

,name,timestamp
0,Matej Kovac,2018-10-07
1,Zana Kepic,2018-10-20
2,Luka Nagode,2018-11-02
3,Nika Zgojznik,2018-11-03
4,Nik Krajnc,2018-11-04
5,Spela Sellak,2018-11-09
6,Ana Cesnjevar,2018-11-11
7,Katarina Bartolj,2018-11-24
8,Gasper Sojar,2018-11-27
9,Damijan Mocnik,2018-11-30
